In [1]:
#%%
from datetime import datetime
import pandas as pd
import numpy as np
import os
import sys
sys.path.append('/home/mes/venv/lib/python2.7/site-packages/')
import xgboost as xgb
## label encoding
import sklearn

SUBSET = True

#%%
DIR_PATH = '../../data/'
train_file = 'train_total.csv'
test_file = 'test_total.csv'

## loading data as Pandas dataframes
train_raw = pd.read_csv(os.path.join(DIR_PATH, train_file), 
                        header='infer', 
                        index_col='id',
                        parse_dates=['timestamp'])
test_raw = pd.read_csv(os.path.join(DIR_PATH, test_file), 
                       header='infer', 
                       index_col='id',
                       parse_dates=['timestamp'])

#%%
## Trim down the sub_area levels to the top 25 and put all others as there
## own separate level
freq_area = np.array(train_raw.loc[:, 'sub_area'].value_counts()[:25].index)

train_raw.loc[~train_raw['sub_area'].isin(freq_area), 'sub_area'] = 'other'
test_raw.loc[~test_raw['sub_area'].isin(freq_area), 'sub_area'] = 'other'

## time features, the timestamp as is makes the xgboost fail
train_raw.loc[:, 'year'] = train_raw.loc[:, 'timestamp'].apply(lambda x: x.strftime('%Y'))
train_raw.loc[:, 'month'] = train_raw.loc[:, 'timestamp'].apply(lambda x: x.strftime('%m'))

test_raw.loc[:, 'year'] = test_raw.loc[:, 'timestamp'].apply(lambda x: x.strftime('%Y'))
test_raw.loc[:, 'month'] = test_raw.loc[:, 'timestamp'].apply(lambda x: x.strftime('%m'))

## This allows the model to run over a subset of the entire data
if SUBSET:
    features = ['month', 'year', 'full_sq', 'life_sq', 'floor', 
                    'max_floor', 'material', 'build_year', 'num_room',
                    'kitch_sq', 'state', 'radiation_km', 'basketball_km',
                    'museum_km'] 
    train = train_raw[features]
    test = test_raw[features]
else:
    train = train_raw.copy()
    test = train_raw.copy()
    features = list(test.columns)

#%%    
## Must encode object columns for the model
for f in train.columns:
    if train[f].dtype=='object':
        print('encoding training feature: {}'.format(f))
        lbl = sklearn.preprocessing.LabelEncoder()
        train.loc[:,f] = lbl.fit_transform(train.loc[:,f])
        
for f in test.columns:
    if test[f].dtype=='object':
        print('encoding test feature: {}'.format(f))
        lbl = sklearn.preprocessing.LabelEncoder()
        test.loc[:,f] = lbl.fit_transform(test.loc[:,f])

#%%
# Convert data frames to numpy arrays
X_train = train.values
Y_train = train_raw['price_doc'].values
X_test = test.values

#%%
# Subset to tune XGB 'num_boost_rounds'
size_ = 7000
X_train_sub, Y_train_sub = X_train[:-size_],  Y_train[:-size_]
X_val, Y_val = X_train[-size_:],  Y_train[-size_:]

dtrain = xgb.DMatrix(X_train, 
                    Y_train, 
                    feature_names=features)
dtrain_sub = xgb.DMatrix(X_train_sub, 
                        Y_train_sub, 
                        feature_names=features)
d_val = xgb.DMatrix(X_val, 
                    Y_val, 
                    feature_names=features)
dtest = xgb.DMatrix(X_test, 
                    feature_names=features)


/home/mes/venv/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


encoding training feature: month
encoding training feature: year
encoding test feature: month
encoding test feature: year


/home/mes/venv/lib/python2.7/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [21]:
#%%
# hyperparameters
xgb_params = {
    'eta': 0.02,
    'max_depth': [5,6],
    'subsample': .8,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}

#Tune the model
#sub_model = xgb.train(xgb_params, 
#                      dtrain_sub, 
#                      num_boost_round=2000,
#                      evals=[(d_val, 'val')],
#                      early_stopping_rounds=20, 
#                      verbose_eval=50)
print(datetime.now())

#cv = xgb.cv(dtrain=dtrain, num_boost_round = 1, nfold = 5, metrics = list("rmse"), 
#            max_depth = 3, eta = .5, objective = "reg:linear")

cv = xgb.cv(dtrain=dtrain, num_boost_round = 1, nfold = 5, params = xgb_params)
            
print(cv)
print(cv)

print(datetime.now())
#%%
# Train the model
#full_model = xgb.train(xgb_params,
#                       dtrain, 
#                       num_boost_round=sub_model.best_iteration,
#                       verbose_eval=20)

# predict the prices from the test data
#y_pred = full_model.predict(dtest)

#%%
# Write them to csv for submission
#submit = pd.DataFrame({'id': np.array(test.index), 'price_doc': y_pred})
#submit.to_csv('submissions/submission_xgb3.csv', index=False)


2017-05-25 14:00:44.090089


XGBoostError: Invalid Parameter format for max_depth expect int but value='[5, 6]'

In [16]:
cv.shape[0]

1

In [18]:
for row in range(cv.shape[0]):
    print cv.iloc[0,:].T

test-rmse-mean     8.384986e+06
test-rmse-std      9.297573e+04
train-rmse-mean    8.384469e+06
train-rmse-std     2.353557e+04
Name: 0, dtype: float64


In [28]:
df = pd.DataFrame([[1,2,3], [4,5,6]])
df

,0,1,2
0,1,2,3
1,4,5,6


In [32]:
for i in range(df.shape[0]):
    print df.iloc[i,:]

0    1
1    2
2    3
Name: 0, dtype: int64
0    4
1    5
2    6
Name: 1, dtype: int64
